In [1]:
%cd implicature

/gws/nopw/j04/ai4er/users/maiush/implicature


In [2]:
from constants import results_path

import pickle, string
import numpy as np
import pandas as pd
import torch as t

In [3]:
models = ["gemma-2-2b", "llama-3.1-8b", "mistral-nemo-12b", "qwen-2-72b"]

data = pd.read_json(f"test_data.jsonl", orient="records", lines=True)
true = (data["implicature"] == "no").to_numpy() * 1

In [4]:
results = pd.DataFrame(columns=["base", "instruct"])
for model in models:
    base = t.load(f"{results_path}/{model}-base/zero_shot.pt", weights_only=True)
    instruct = t.load(f"{results_path}/{model}-instruct/zero_shot.pt", weights_only=True)

    base = (base[:, 0, :].argmax(dim=-1).numpy() == true).mean()
    instruct = (instruct[:, 0, :].argmax(dim=-1).numpy() == true).mean()

    results.loc[model] = [base, instruct]
results

,base,instruct
gemma-2-2b,0.623333,0.691667
llama-3.1-8b,0.576667,0.805000
mistral-nemo-12b,0.790000,0.818333
qwen-2-72b,0.803333,0.858333


In [5]:
def parse_answers(answers):
    answers = [s.lower().translate(str.maketrans('', '', string.punctuation)) for s in answers]
    out = []
    for answer in answers:
        if answer.startswith("no"): out.append(1)
        elif answer.startswith("yes"): out.append(0)
        else: out.append(-1)
    return np.array(out)
results = pd.DataFrame(columns=["base", "instruct"])
for model in models:
    with open(f"{results_path}/{model}-base/zero_shot_answers.pkl", "rb") as f:
        base_answers = pickle.load(f)
        base = parse_answers(base_answers)
    with open(f"{results_path}/{model}-instruct/zero_shot_answers.pkl", "rb") as f:
        instruct_answers = pickle.load(f)
        instruct = parse_answers(instruct_answers)
    mask = np.argwhere(base != -1).flatten()
    base = (base[mask] == true[mask]).mean()
    instruct = (instruct[mask] == true[mask]).mean()
    results.loc[model] = [base, instruct]
results

,base,instruct
gemma-2-2b,0.587219,0.696028
llama-3.1-8b,0.587219,0.787565
mistral-nemo-12b,0.797927,0.803109
qwen-2-72b,0.803819,0.848958
